<a href="https://colab.research.google.com/github/hanhluukim/replication-topic-modelling-in-embedding-space/blob/main/BERT_Embedding_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 41.4 MB/s 
     |████████████████████████████████| 6.6 MB 10.5 MB/s 
     |████████████████████████████████| 84 kB 1.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=dfb5eae33f315f160fc6c46bac2ab9ec3814b84f047a940be64b6383a330eaec
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [3]:
# using bert-model to create static the word-embedding
# https://datascience.stackexchange.com/questions/85566/how-pre-trained-bert-model-generates-word-embeddings-for-out-of-vocabulary-words

from distutils.util import split_quoted
import torch
from transformers import BertTokenizer, BertModel
from sklearn.datasets import fetch_20newsgroups
import re
from nltk.tokenize import word_tokenize
import nltk
import string
nltk.download('punkt')

#with open('src/stops.txt', 'r') as f:
#    stops = f.read().split('\n')

from stop_words import get_stop_words
stop_words = get_stop_words('en')
    

newsgroups_train = fetch_20newsgroups(subset='train')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [4]:
# read train_data from 20newsgroups
def read_raw_documents():
    raw_documents = []
    raw_labels = []
    for i in range(0,len(newsgroups_train.data)):
        raw_documents.append(newsgroups_train.data[i])
        raw_labels.append(newsgroups_train.target[i])
    return raw_documents, raw_labels

def simple_preprocess(raw_documents):
    def only_letters(tested_string):
        for letter in tested_string:
            if letter not in "abcdefghijklmnopqrstuvwxyz":
                return False
        return True
    def clean_doc_for_bert(doc): 
        doc = doc.replace(">","").lower()
        word_list = word_tokenize(doc) #only using empty space and punctation for tokenization
        cleaned = []
        for w in word_list:
            if w not in stop_words:
                if w in string.punctuation or only_letters(w): #using only character from punctation and alpha characters
                    if w in string.punctuation or len( set(w) ) > 1: #punctation with len 1 allowed but alpha word must be longer then 1
                        cleaned.append( w)
        return " ".join(cleaned), cleaned  #save doc in string and in token-list         
       
    cleaned_documents = []
    for doc in raw_documents:
        doc_in_string, doc_in_token_list = clean_doc_for_bert(doc)
        cleaned_documents.append(doc_in_string)
    return cleaned_documents

def transform_to_sentences_with_labels():
    # we will not use labels
    sentences_with_labels = []
    return sentences_with_labels

def fine_tune_bert():
    # should to be trained?
    # no, because for topic modelling, that is usupervised problem. We just find topics for the documents
    # topic modelling no targets
    return True

def transform_to_sentences(docs): #no labels
    data_as_sentences = []
    for doc in docs:
      for sent in doc.split(". "): #make sentences
        updated_sent = " ".join([t for t in sent.split(" ") if len(t) > 1])
        if len(updated_sent.split(" ")) > 1:
            data_as_sentences.append(updated_sent)
        else:
            if updated_sent not in data_as_sentences:
                data_as_sentences.append(updated_sent)
    return data_as_sentences

def split_long_sentence(splitted_sent, given_len):
    subsents = []
   #for i in range(0,len(splitted_sent), given_len):
    i=0
    while i < len(splitted_sent): 
        if i == 0:
            sub = " ".join(splitted_sent[i:i+given_len])
            subsents.append(sub)
            i = i + given_len
        if i!=0:
            j = i + given_len - 5 #windown 5
            if j + given_len <= len(splitted_sent):
                sub = " ".join(splitted_sent[j:j + given_len])
                subsents.append(sub)
            else:
                sub = " ".join(splitted_sent[j:])
                if len(sub)>1:
                    subsents.append(sub)
            i = j + given_len
    return subsents

def handle_long_sentences(sentences, given_len):
    # overlapped splitting sentence windown 5
    subsents = []
    deleted_long_sents = []
    for sent in sentences:
        splitted_sent = sent.split(" ")
        if len(splitted_sent) > given_len:
          long_sent_subsents = split_long_sentence(splitted_sent, given_len)
          subsents.extend(long_sent_subsents)
          deleted_long_sents.append(sent)
    # update sentences: remove and add subsents
    for del_sent in deleted_long_sents:
        sentences.remove(del_sent)
    for add_sent in subsents:
        sentences.append(add_sent)
    return sentences

def create_marked_senteces(sentences):
    return ['[CLS] ' + sent.strip() + ' [SEP]' for sent in sentences]
def save_sents_to_txt(shorted_sentences):
    with open(r'./bert_sentences.txt', 'w') as fp:
      for sent in shorted_sentences:
          # write each item on a new line
          fp.write(f'{sent} \n')
      print('saving sentences from bert-processing')
    return True

# **Datenvorbereitung**

In [5]:
print("reading data:...")
raw_documents, _ = read_raw_documents()
print(len(raw_documents))
print("preprocess data:...")
preprocessed_docs = simple_preprocess(raw_documents)
print(len(preprocessed_docs))
print("transform to sentences:...")
sentences = transform_to_sentences(preprocessed_docs)
print("split sentences to 128 tokens:...")
shorted_sentences =  handle_long_sentences(sentences, 128)
shorted_sentences = create_marked_senteces(shorted_sentences)
# write sentences to txt files
with open(r'bert_sentences.txt', 'w') as fp:
    for sent in shorted_sentences:
        # write each item on a new line
        fp.write(f'{sent} \n')
    print('saving sentences from bert-processing')
print("finished: ...")



reading data:...
11314
preprocess data:...
11314
transform to sentences:...
split sentences to 128 tokens:...
saving sentences from bert-processing
finished: ...


# **Embeddings**

In [6]:
from transformers import BertTokenizerFast, BertTokenizer
tokenizer_ = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
from transformers import BertTokenizerFast

tokenizerfast = BertTokenizerFast.from_pretrained('bert-base-uncased')
#tokens = t('word embeddings are vectors', add_special_tokens=False, return_attention_mask=False, return_token_type_ids=False)


# **Information nach dem Tokenizer**

In [112]:
def tokenizer_for_a_sent(sent, tokenizer):
  this_sent_tokenizer = tokenizer(sent)
  # index of token in the vocabulary
  indexed_tokens = this_sent_tokenizer.input_ids
  segments_ids = [1] * len(indexed_tokens)
  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  return tokens_tensor, segments_tensors
  
def reform_token_embeddings_of_sentence(full_outputs):
  hidden_states = outputs[2]
  token_embeddings = torch.stack(hidden_states, dim=0)
  #print(token_embeddings.shape)
  token_embeddings = torch.squeeze(token_embeddings, dim=1) # size= (n_hidden_layers, n_tokens, 768)
  #print(token_embeddings.shape)
  token_embeddings = token_embeddings.permute(1,0,2) # size= (n_tokens, n_hidden_layers, 768)
  #print(token_embeddings.shape)
  return token_embeddings 

def get_token_embeddings(reformed_token_embeddings):
  # using sum four last layers
  token_vecs_sum = []
  print(f'get-token-embedding-function: {reformed_token_embeddings.shape}')
  for i, token in enumerate(reformed_token_embeddings): 
    sum_vec = torch.sum(token[-4:], dim=0)
    token_vecs_sum.append(sum_vec)
    #print(f'original {token.shape} and token-emb after sum {sum_vec.shape}')
  return token_vecs_sum # size: n_tokens: 768

def get_subwords_embeddings_of_word(bert_unique_token_id, tokenized_indices, tokens_embeddings):
    belongging_embeddings_of_word = []
    for idx, tokenizer_idx in enumerate(tokenized_indices):
        if tokenizer_idx == bert_unique_token_id:
            belongging_embeddings_of_word.append(tokens_embeddings[idx])
    return torch.stack(belongging_embeddings_of_word, dim=0)

def get_unique_embedding(embeddings=None, methode="mean"):
    #print(embeddings[0].shape)
    if methode == "mean":
        if embeddings.shape[0] == 1:
          return torch.squeeze(embeddings, dim=0)
        else:
          mean_embedding = torch.mean(embeddings, dim=0) #torch.tensor([embeddings])#.mean()
          return mean_embedding

def need_to_update(sent_tokens_ids):
    special_ids = [101, 102] #of CLS and SEP
    for e in sent_tokens_ids:
        if e in special_ids:
            sent_tokens_ids.remove(e)
    return sent_tokens_ids
  
def get_multiple_embeddings_for_words_in_sent(sent_tokens_ids, sent_outputs_tokens_embeddings):
    # a word can be one time oder multiple times in a sentence
    print(f'tokens-ids in get_multiple_embeddings_: {sent_tokens_ids}')
    sent_tokens_ids = need_to_update(sent_tokens_ids)
    multiple_words_embeddings = []
    unique_words_ids = list(set(sent_tokens_ids))
    for unique_id in unique_words_ids:
        belong_embeddings = get_subwords_embeddings_of_word(unique_id, sent_tokens_ids, sent_outputs_tokens_embeddings)
        print(f'word-id: {unique_id} - beling-embeddings shape: {belong_embeddings.shape}')
        # mean of belonging_embeddings to get embedding of whole word
        word_embedding = get_unique_embedding(belong_embeddings, "mean")
        print(f'mean-word-id {unique_id} word-embedding {word_embedding.shape}')
        multiple_words_embeddings.append(word_embedding)
        print("----------------------------------------------------------")
    return torch.stack(multiple_words_embeddings, dim=0)

def get_indices_of_word_in_original_sent(word, splitted_original_sent):
    indices = []
    for i, e in enumerate(splitted_original_sent):
        if e == word:
            indices.append(i)
    return indices

def get_final_words_embeddings_in_sent(original_sent, sent_tokens_ids, sent_outputs_tokens_embeddings):
    #import numpy as np
    not_unique_words_embeddings = get_multiple_embeddings_for_words_in_sent(sent_tokens_ids, sent_outputs_tokens_embeddings)
    print(f'total found embeddings in sent: {not_unique_words_embeddings.shape}')
    original_words_list = original_sent.split(" ")
    words_embeddings_in_sent_dict = {}
    for word in set(original_words_list):
        if word not in ['[CLS]', '[SEP]']:
          word_indices = get_indices_of_word_in_original_sent(word, original_words_list)
          print(f'word---- {word} ---- indices in original sent: {word_indices}')
          # a word can have different-word-embeddings in the sentence, because a word can occur multple times
          # each occurance has a different embedding for this word
          different_occurrences_embeddings_of_word = not_unique_words_embeddings[word_indices]
          print(f'test: {different_occurrences_embeddings_of_word.shape}')
          mean_unique_word_embedding = get_unique_embedding(torch.tensor(different_occurrences_embeddings_of_word), "mean")
          words_embeddings_in_sent_dict[word] = mean_unique_word_embedding
    return words_embeddings_in_sent_dict
def save_embeddings_in_sent_to_text(words_embeddings_in_sent_dict):
    with open(r'./bert_sent_words_embeddings.txt', 'w') as fp:
      for word, vector in words_embeddings_dict_in_sent.items():
          # write each item on a new line
          fp.write(f'{word}\t')
          for e in vector.tolist():
            fp.write(f'{e} ')
          fp.write("\n")
      print('saving embeddings')
    return True

# **BertTokenizerFast**

In [114]:
ex_sent = shorted_sentences[0]
print(ex_sent)
print(ex_sent.split(" "))

sent_tokenizer = tokenizerfast(ex_sent, add_special_tokens=False)
#print(tokens.word_ids())

# index of token in the vocabulary
indexed_tokens_ids = sent_tokenizer.input_ids
segments_ids = [1] * len(indexed_tokens_ids)
print(f'tokens-ids: {indexed_tokens_ids}')
print(f'len-tokens: {len(indexed_tokens_ids)}')

tokens_ids_with_belonging_information = sent_tokenizer.word_ids()
print(tokens_ids_with_belonging_information)
print(len(tokens_ids_with_belonging_information))

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens_ids])
segments_tensors = torch.tensor([segments_ids])
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]
    print(len(hidden_states))
    token_embeddings = torch.stack(hidden_states, dim=0)
    print(token_embeddings.size())
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    print(token_embeddings.size())
    #print(hidden_states.shape)
# using BertTokenizerFast
    reformed = reform_token_embeddings_of_sentence(outputs)
    sent_tokens_embeddings = get_token_embeddings(reformed)
    print(f'number of found embeddings: {len(sent_tokens_embeddings)}')
    words_embeddings_dict_in_sent = get_final_words_embeddings_in_sent(ex_sent, tokens_ids_with_belonging_information, sent_tokens_embeddings)
    save_embeddings_in_sent_to_text(words_embeddings_dict_in_sent)

[CLS] lerxst thing subject car organization university maryland college park lines wondering anyone enlighten car saw day [SEP]
['[CLS]', 'lerxst', 'thing', 'subject', 'car', 'organization', 'university', 'maryland', 'college', 'park', 'lines', 'wondering', 'anyone', 'enlighten', 'car', 'saw', 'day', '[SEP]']
tokens-ids: [101, 3393, 2099, 2595, 3367, 2518, 3395, 2482, 3029, 2118, 5374, 2267, 2380, 3210, 6603, 3087, 4372, 7138, 2368, 2482, 2387, 2154, 102]
len-tokens: 23
[0, 1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 13, 13, 14, 15, 16, 17]
23
13
torch.Size([13, 1, 23, 768])
torch.Size([13, 23, 768])
get-token-embedding-function: torch.Size([23, 13, 768])
number of found embeddings: 23
tokens-ids in get_multiple_embeddings_: [0, 1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 13, 13, 14, 15, 16, 17]
word-id: 0 - beling-embeddings shape: torch.Size([1, 768])
mean-word-id 0 word-embedding torch.Size([768])
----------------------------------------------------------
word-id: 1 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [108]:
words_embeddings_dict_in_sent.keys()

dict_keys(['lines', 'park', 'wondering', '[CLS]', 'car', 'organization', 'college', 'lerxst', 'subject', 'thing', 'anyone', 'enlighten', 'saw', 'day', '[SEP]', 'university', 'maryland'])

# **BertTokenizer**

In [50]:


# using BertTokenizer
tokens = tokenizer_.tokenize(ex_sent)
print(tokens)
print(len(tokens))
indexed_tokens = tokenizer_.convert_tokens_to_ids(tokens)
print(indexed_tokens)
segments_ids = [1] * len(tokens)
print(segments_ids)
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]
    print(len(hidden_states))
    #token_embeddings = torch.stack(hidden_states, dim=0)
    #print(token_embeddings.size())
    #token_embeddings = torch.squeeze(token_embeddings, dim=1)
    #print(token_embeddings.size())
    #print(hidden_states.shape)
# using BertTokenizerFast
    reformed = reform_token_embeddings_of_sentence(outputs)
    sent_tokens_embeddings = get_token_embeddings(reformed)

[CLS] lerxst thing subject car organization university maryland college park lines wondering anyone enlighten car saw day [SEP]
['[CLS]', 'le', '##r', '##x', '##st', 'thing', 'subject', 'car', 'organization', 'university', 'maryland', 'college', 'park', 'lines', 'wondering', 'anyone', 'en', '##light', '##en', 'car', 'saw', 'day', '[SEP]']
23
[101, 3393, 2099, 2595, 3367, 2518, 3395, 2482, 3029, 2118, 5374, 2267, 2380, 3210, 6603, 3087, 4372, 7138, 2368, 2482, 2387, 2154, 102]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
13
torch.Size([13, 1, 23, 768])
torch.Size([13, 23, 768])
torch.Size([23, 13, 768])
get-token-embedding-function: torch.Size([23, 13, 768])
original torch.Size([13, 768]) and token-emb after sum torch.Size([768])
original torch.Size([13, 768]) and token-emb after sum torch.Size([768])
original torch.Size([13, 768]) and token-emb after sum torch.Size([768])
original torch.Size([13, 768]) and token-emb after sum torch.Size([768])
original torch.Si

In [10]:
def embeddings_using_tokenizer():
  return True
def embeddings_using_tokenizerfast():
  return True

In [ ]:
for i in range(0,len(sentences)):
  sent = shorted_sentences[i]
  tokens_tensor, segments_tensors = tokenizer_for_a_sent(sent)
  """
  tk = tokenizer(sent)
  print(tk.input_ids)
  print(tk.token_type_ids)
  print(tk.word_ids())
  print(len(tk.word_ids()))
  print(tokenizer.tokenize(sent))
  print(len(tokenizer.tokenize(sent)))
  print("---------------------------------------")
  """
  with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]
    #print(hidden_states)
    print(len(hidden_states))
    print(hidden_states[0].shape)